In [ ]:
import pandas as pd
import numpy as np
from skimage.feature import peak_local_max
from sklearn.cluster import DBSCAN
import skimage
import baseline_correction
from read_chroma import read_chromato_and_chromato_cube
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
import projection
import plot
import read_chroma


In [ ]:
# select method
# method = "peak_local_max"
# method = "persistence_homology"
method = 'DOG'

In [ ]:


file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
# estimation with filtering_factor_estimation.ipynb
mod_time = 1.7
eps = 0.2

thresold_abs = 0.2

# estimation with estimation_persistence.ipynb
min_persitence = 0.02 # = threshold_relatif

In [ ]:
# open csv
df = pd.read_csv(f'figs/coordinates_{method}.csv')
coordinates_all_mass = df.to_numpy()

In [ ]:
coordinates_all_mass = np.delete(coordinates_all_mass, 0, -1) # supp l'info de la mass
len(coordinates_all_mass)

In [ ]:
#clustering
#TODO scaled value before DBSCAN
coordinates_all_mass_scaled = StandardScaler().fit_transform(coordinates_all_mass)

clustering = DBSCAN(eps=eps, min_samples=5).fit(coordinates_all_mass_scaled[:, :2])
np.max(clustering.labels_)


In [ ]:
# nombre de clusters détectés (sans le label -1)
#TODO a corriger ds le clustering , enelever les points de bruit + construire la liste des clusters sans les bruits
n_clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
# initialisation de la liste des clusters
clusters = [[] for _ in range(n_clusters)]


for i, (t1, t2) in enumerate(coordinates_all_mass):
    label = clustering.labels_[i]
    if label != -1: # enleve les points de bruit
        clusters[label].append([t1, t2])

print("nb de clusters", len(clusters))
n_noise = np.sum(clustering.labels_ == -1)
print(f"Nombre de points considérés comme du bruit : {n_noise}")

In [ ]:
# Coordinates of the geometrical center point in the cluster for every clusters
coordinates = []
for cluster in clusters:
    if not cluster:
        print("Erreur, cluster vide")
    elif (len(cluster) > 1):
        # point le + intense:
        # coord = cluster[np.argmax(np.array([chromato[coord[0], coord[1]] for coord in cluster]))] 

        arr = np.array(cluster)
        # centre geometrique du cluster
        median_x = np.median(arr[:, 0]) 
        median_y = np.median(arr[:, 1])
        coord=[round(median_x),round(median_y)]
    else:
        coord = cluster[0]

    coordinates.append(coord)
peaks_coordinates = np.array(coordinates)

peaks_coordinates

In [ ]:
tic_chromato, time_rn, spectra_obj = read_chroma.read_chroma(file_path, mod_time, None)
chromato_tic_preprocessed = baseline_correction.chromato_reduced_noise(tic_chromato)

coordinates_in_chromato = projection.matrix_to_chromato(
    peaks_coordinates, 
    time_rn, 
    mod_time, 
    chromato_tic_preprocessed.shape
    )
plot.visualizer(
    (chromato_tic_preprocessed, time_rn),
    mod_time,
    title=method,
    log_chromato=False,
    points=coordinates_in_chromato,
    save=True)  